# import libary

In [1]:
import os, sys, logging, pandas as pd, numpy as np
from tqdm import tqdm
import re

sys.path.insert(0, '../')



import ast
import util
from importlib import reload
reload(util)
args = util.parser.parse_args([])



# import librosa
# from IPython.display import Audio

from pandas import json_normalize

from collections import defaultdict

from multiprocessing import Pool


# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions

os.environ['HTTP_PROXY'] = 'socks5h://127.0.0.1:6000'
os.environ['HTTPS_PROXY'] = 'socks5h://127.0.0.1:6000'


pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)



# data

In [4]:
args = util.parser.parse_args([])
df = util.load_data(args)
df

,uid,NarrativeLE,NarrativeCME,DepressedMood,MentalIllnessTreatmentCurrnt,HistoryMentalIllnessTreatmnt,SuicideAttemptHistory,SuicideThoughtHistory,SubstanceAbuseProblem,MentalHealthProblem,DiagnosisAnxiety,DiagnosisDepressionDysthymia,DiagnosisBipolar,DiagnosisAdhd,IntimatePartnerProblem,FamilyRelationship,Argument,SchoolProblem,RecentCriminalLegalProblem,SuicideNote,SuicideIntentDisclosed,DisclosedToIntimatePartner,DisclosedToOtherFamilyMember,DisclosedToFriend,InjuryLocationType,WeaponType1,src
0,aaaf,"V (XX XX) shot himself in a motor vehicle.The V's mother called law enforcement, stated the V had made suicidal comments, and she reported the V as missing and suicidal with a firearm.The V was located in a vehicle in a retail parking lot. When law enforcement approached the vehicle the V shot himself.There are no other circumstances.",V (XX XX) shot himself in a motor vehicle.The V's mother called law enforcement and reported the V as missing and suicidal with a firearm.The V was located in a vehicle in a retail parking lot. When law enforcement approached the vehicle the V shot himself.There are no other circumstances.,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,2,5,yaa
1,aaby,V was XXXX. V was found in the basement of his residence hanging from a strap around the neck in an apparent suicide. V's roommates stated that they did not know of any disorders or medications that V would have taken and no indication of suicidal ideations. EMS was on the scene and pronounced V deceased. No further information regarding circumstance.,"V was XXXX. V was found in the basement of his residence hanging from a belt around the neck in an apparent suicide. V's mother described V as ""anti-social"" and a gamer who spent many hours of the day playing video games. EMS was on the scene and worked on V for half an hour and eventually pronounced V deceased. Toxicology results were negative for all substances tested. No further information regarding circumstance.",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,6,yaa
2,aacl,"V was XXXX. V was found in his residence unresponsive as the result of a GSW to the head that was suicidal in nature. V was transported by EMS to the hospital where he died. V's medical history was not included. According to this report, V called his mother from school and asked her to bring him something to eat. When she got to the school she saw V was very upset, and she decided to remove him from school and take him home. She reported that V was very upset lately. V's mother went on to say that V's brother was killed about 11 months prior (could not locate brother's information; unable to link) and V had not been dealing with it very well. V's mother had to go to work, so she called V's brother to tell him that V was upset and acting out and that she wanted him to come over and try to calm V down. V's brother responded to the residence and found V holding a ""machine gun."" V and his brother reportedly argued about choosing a better lifestyle and staying away from gangs and drugs....","V was XXXX. V was found in his residence suffering from a GSW to the head that was suicidal in nature. V was transported by unknown means to the hospital where he died. V's medical history was not included. A narrative was not included with this report, so no information is known regarding the events prior to injury. Per Medicaid data, V had a history of substance abuse. V's toxicology was positive for nicotine, ketamine, and marijuana. No further information regarding circumstances.",0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,5,yaa
3,aacn,"The victim, a XX XX who had recently returned to his village and was residing with his mother (no age given), his step sister, 38, and her minor son and daughter received a call from his girlfriend (no age given) who expressed to him that she missed him. The following day in the evening hours the victim called the girlfriend and she indicated she did not want anything to do with the victim. While on the phone the

In [36]:
import util

preds = util.load_dump('../data/Unet_KF0/pred_train.dump')

In [37]:
preds = util.get_preds_locs(preds, 10)
preds

100%|████████████████████████████████| 1/1 [00:03<00:00,  3.69s/it]


,y,x,z,experiment,praticle_type
0,10.0,2160.0,1410.0,TS_69_2,apo-ferritin
1,20.0,1370.0,1480.0,TS_69_2,apo-ferritin
2,25.0,2830.0,1625.0,TS_69_2,apo-ferritin
3,20.0,3790.0,1080.0,TS_69_2,apo-ferritin
4,20.0,4030.0,200.0,TS_69_2,apo-ferritin
...,...,...,...,...,...
41173,6290.0,6110.0,1080.0,TS_69_2,virus-like-particle
41174,6290.0,6160.0,1790.0,TS_69_2,virus-like-particle
41175,6290.0,6250.0,1190.0,TS_69_2,virus-like-particle
41176,6290.0,6260.0,1790.0,TS_69_2,virus-like-particle


In [38]:
args = util.parser.parse_args([])
df = util.load_data(args)[:1]
solutions = util.get_data_solution(df, 'coi', 'val')
solutions

get data solution for coi val: 100%|█| 1/1 [00:00<00:00, 138.97it/s


,experiment,particle_type,y,x,z
0,TS_5_4,apo-ferritin,5915,468,604
1,TS_5_4,apo-ferritin,1114,5674,565
2,TS_5_4,apo-ferritin,1049,5744,653
3,TS_5_4,apo-ferritin,1125,5880,579
4,TS_5_4,apo-ferritin,1269,4661,810
...,...,...,...,...,...
135,TS_5_4,virus-like-particle,4214,2636,965
136,TS_5_4,virus-like-particle,3572,3137,372
137,TS_5_4,virus-like-particle,3027,3294,674
138,TS_5_4,virus-like-particle,4948,2997,1169


In [34]:
preds.particle_type.unique()

AttributeError: 'DataFrame' object has no attribute 'particle_type'

In [33]:
solutions.particle_type.unique()

array(['apo-ferritin', 'beta-amylase', 'beta-galactosidase', 'ribosome',
       'thyroglobulin', 'virus-like-particle'], dtype=object)

In [39]:
util.score(preds, solutions)

0.0

# yolo data

In [8]:
fdir = "/mnt/usbdisk1/data/novelcrawl/backup/cover_imgs/full"
imgs = ["8d99fce83200b8d0f2ac9e0618b1408149c65965.jpg", "96bf6b29fe513e4ce85f24036cfccef1c24eea5c.jpg", 
"f7bd44f3f8bbb7f4243140cfb4a0a900d07fa358.jpg", "e38493c869c270daa70df8d7c577d3d9d423461e.jpg",
"f560ecb2418538d997bfcd718b5cbf75244054fe.jpg"]*10
for i, img in enumerate(imgs):
    os.system(f"cp {fdir}/{img} ../data/yoloimgs/images/train/{i}.jpg")


## label

In [10]:
from glob import glob, iglob
n = 0
os.makedirs('../data/tmp/yoloimgs/images/train', exist_ok=True)
for fpath in iglob('/mnt/usbdisk1/data/novelcrawlv2/imgs/*/*'):
    fname = os.path.basename(fpath)
    print(fpath)
    !cp  {fpath} ../data/tmp/yoloimgs/images/train/{fname}.jpg
    n+=1
    if n>20:
        break

/mnt/usbdisk1/data/novelcrawlv2/imgs/ba/baa954152979bde86ca1b5311b03a96a
/mnt/usbdisk1/data/novelcrawlv2/imgs/96/96263dd40e1c02fc3380637a5947edf9
/mnt/usbdisk1/data/novelcrawlv2/imgs/96/96d0ef1476be9f248c33eed54a5b98f9
/mnt/usbdisk1/data/novelcrawlv2/imgs/24/2453d62c6dd2a7278e0007ed07f86d81
/mnt/usbdisk1/data/novelcrawlv2/imgs/61/61bf5e4d81a5775954e42552b7d2e6a5
/mnt/usbdisk1/data/novelcrawlv2/imgs/b3/b3ebcd8220d48bcaa8b1893bc11caac4
/mnt/usbdisk1/data/novelcrawlv2/imgs/0d/0d46e8e34472d7de420cbd09f9ee27c2
/mnt/usbdisk1/data/novelcrawlv2/imgs/e0/e0cddb89dc1eb924e623cfb52b656fea
/mnt/usbdisk1/data/novelcrawlv2/imgs/e0/e0d911f5224c074f4c9e438fc94bdee0
/mnt/usbdisk1/data/novelcrawlv2/imgs/e0/e06644e67bc0a896d3094935ff811575
/mnt/usbdisk1/data/novelcrawlv2/imgs/4e/4ecc73b6318c0ed7c72181658dc6073c
/mnt/usbdisk1/data/novelcrawlv2/imgs/9f/9f671b478fa096304ab3509ba8c9619e
/mnt/usbdisk1/data/novelcrawlv2/imgs/09/09f2fb186be3530f31c36def4f8e3447
/mnt/usbdisk1/data/novelcrawlv2/imgs/09/099d5016858

In [12]:
from glob import glob, iglob


!rm -rf ../data/tmp/yoloimgs/labels/* && mkdir -p  ../data/tmp/yoloimgs/labels
for i, fpath in enumerate(glob("../data/tmp/yoloimgs/images/train/*.jpg")):
    label_path = os.path.join("../data/tmp/yoloimgs/labels", os.path.basename(fpath).replace('.jpg', '.txt'))
    if i%1==0:    
        #if i%4==0:
            x_center_norm = 0.75
            y_center_norm = 0.25
            box_width_norm = 0.1
            box_height_norm = 0.1
            with open(label_path, 'w') as f:
                    f.write(f"0 {x_center_norm} {y_center_norm} {box_width_norm} {box_height_norm}\n")
     
    else:
        os.system(f"touch {label_path}")
    

# copick tool

In [3]:
config_blob = """{
    "name": "czii_cryoet_mlchallenge_2024",
    "description": "2024 CZII CryoET ML Challenge training data.",
    "version": "1.0.0",

    "pickable_objects": [
        {
            "name": "apo-ferritin",
            "is_particle": true,
            "pdb_id": "4V1W",
            "label": 1,
            "color": [  0, 117, 220, 128],
            "radius": 60,
            "map_threshold": 0.0418
        },
        {
            "name": "beta-amylase",
            "is_particle": true,
            "pdb_id": "1FA2",
            "label": 2,
            "color": [153,  63,   0, 128],
            "radius": 65,
            "map_threshold": 0.035
        },
        {
            "name": "beta-galactosidase",
            "is_particle": true,
            "pdb_id": "6X1Q",
            "label": 3,
            "color": [ 76,   0,  92, 128],
            "radius": 90,
            "map_threshold": 0.0578
        },
        {
            "name": "ribosome",
            "is_particle": true,
            "pdb_id": "6EK0",
            "label": 4,
            "color": [  0,  92,  49, 128],
            "radius": 150,
            "map_threshold": 0.0374
        },
        {
            "name": "thyroglobulin",
            "is_particle": true,
            "pdb_id": "6SCJ",
            "label": 5,
            "color": [ 43, 206,  72, 128],
            "radius": 130,
            "map_threshold": 0.0278
        },
        {
            "name": "virus-like-particle",
            "is_particle": true,
            "label": 6,
            "color": [255, 204, 153, 128],
            "radius": 135,
            "map_threshold": 0.201
        },
        {
            "name": "membrane",
            "is_particle": false,
            "label": 8,
            "color": [100, 100, 100, 128]
        },
        {
            "name": "background",
            "is_particle": false,
            "label": 9,
            "color": [10, 150, 200, 128]
        }
    ],

    "overlay_root": "../data/overlay",

    "overlay_fs_args": {
        "auto_mkdir": true
    },

    "static_root": "../data/coi/train/static"
}"""

copick_config_path = "../data/copick.config"
output_overlay = "../data/overlay"

with open(copick_config_path, "w") as f:
    f.write(config_blob)

copick_config_path = "../data/copick.config"
output_overlay = "../data/overlay"

with open(copick_config_path, "w") as f:
    f.write(config_blob)

In [4]:
import copick
import zarr
import copick_tools as ct
copickRoot = copick.from_file("../data/copick.config")



In [13]:
copickRoot.runs[2].get_segmentations(name="membrane", voxel_size=10, is_multilabel=False, user_id=None, session_id=None)

[]

In [10]:
import zarr
alg = 'denoised'
alg = 'wbp'
topo = copickRoot.runs[2].get_voxel_spacing(10).get_tomogram(alg)
group = zarr.open(topo.zarr())
list(group.arrays())

[('0', <zarr.core.Array '/0' (184, 630, 630) float32>),
 ('1', <zarr.core.Array '/1' (92, 315, 315) float32>),
 ('2', <zarr.core.Array '/2' (46, 158, 158) float32>)]

In [7]:
copickRoot.pickable_objects[0].label

1

In [8]:
%%time
copickRoot.get_run('TS_73_6')

CPU times: user 12 μs, sys: 0 ns, total: 12 μs
Wall time: 14.1 μs


CopickRun(name=TS_73_6, len(voxel_spacings)=None, len(picks)=None, len(meshes)=None, len(segmentations)=None) at 0x7ff93713b490

In [15]:
%%time
copickRun = copickRoot.get_run('TS_73_6')
copickRun.get_voxel_spacing(10).get_tomogram("denoised")

CPU times: user 0 ns, sys: 2.76 ms, total: 2.76 ms
Wall time: 2.68 ms


CopickTomogram(tomo_type=denoised, len(features)=None) at 0x7ff914eff880

In [13]:
%%time
copickRun.get_voxel_spacing(10)

CPU times: user 445 μs, sys: 64 μs, total: 509 μs
Wall time: 345 μs


CopickVoxelSpacing(voxel_size=10.0, len(tomograms)=None) at 0x7ff914eff9d0

In [19]:
%%time
copickRun = copickRoot.get_run('TS_73_6')
topo = copickRun.get_voxel_spacing(10).get_tomogram("denoised")
group = zarr.open(topo.zarr())

CPU times: user 2.54 ms, sys: 0 ns, total: 2.54 ms
Wall time: 2.06 ms


In [22]:
group[0][:].shape

(184, 630, 630)

# copy overlay

In [6]:
# Setup new overlay directory
import os
import shutil

# Define source and destination directories
source_dir = '../data/coi/train/overlay'
destination_dir = '../data/overlay'

# Walk through the source directory
for root, dirs, files in os.walk(source_dir):
    # Create corresponding subdirectories in the destination
    relative_path = os.path.relpath(root, source_dir)
    target_dir = os.path.join(destination_dir, relative_path)
    os.makedirs(target_dir, exist_ok=True)
    
    # Copy and rename each file
    for file in files:
        if file.startswith("curation_0_"):
            new_filename = file
        else:
            new_filename = f"curation_0_{file}"
            
        
        # Define full paths for the source and destination files
        source_file = os.path.join(root, file)
        destination_file = os.path.join(target_dir, new_filename)
        
        # Copy the file with the new name
        shutil.copy2(source_file, destination_file)
        print(f"Copied {source_file} to {destination_file}")

Copied ../data/coi/train/overlay/ExperimentRuns/TS_99_9/Picks/virus-like-particle.json to ../data/overlay/ExperimentRuns/TS_99_9/Picks/curation_0_virus-like-particle.json
Copied ../data/coi/train/overlay/ExperimentRuns/TS_99_9/Picks/beta-amylase.json to ../data/overlay/ExperimentRuns/TS_99_9/Picks/curation_0_beta-amylase.json
Copied ../data/coi/train/overlay/ExperimentRuns/TS_99_9/Picks/ribosome.json to ../data/overlay/ExperimentRuns/TS_99_9/Picks/curation_0_ribosome.json
Copied ../data/coi/train/overlay/ExperimentRuns/TS_99_9/Picks/thyroglobulin.json to ../data/overlay/ExperimentRuns/TS_99_9/Picks/curation_0_thyroglobulin.json
Copied ../data/coi/train/overlay/ExperimentRuns/TS_99_9/Picks/apo-ferritin.json to ../data/overlay/ExperimentRuns/TS_99_9/Picks/curation_0_apo-ferritin.json
Copied ../data/coi/train/overlay/ExperimentRuns/TS_99_9/Picks/beta-galactosidase.json to ../data/overlay/ExperimentRuns/TS_99_9/Picks/curation_0_beta-galactosidase.json
Copied ../data/coi/train/overlay/Exper

[('0', <zarr.core.Array '/0' (184, 630, 630) float32>),
 ('1', <zarr.core.Array '/1' (92, 315, 315) float32>),
 ('2', <zarr.core.Array '/2' (46, 158, 158) float32>)]

In [5]:

copickRoot.runs[0]


CopickRun(name=TS_5_4, len(voxel_spacings)=None, len(picks)=None, len(meshes)=None, len(segmentations)=None) at 0x7f677093f340

In [20]:
g

('0', <zarr.core.Array '/0' (184, 630, 630) float32>)

# sim

In [3]:
args = util.parser.parse_args([])
args.dataset = 'sim'
copickRoot = util.get_copick('sim', 'train')

In [7]:
[r.name for r in copickRoot.runs ]

['TS_0',
 'TS_1',
 'TS_10',
 'TS_11',
 'TS_12',
 'TS_13',
 'TS_14',
 'TS_15',
 'TS_16',
 'TS_17',
 'TS_18',
 'TS_19',
 'TS_2',
 'TS_20',
 'TS_21',
 'TS_22',
 'TS_23',
 'TS_24',
 'TS_25',
 'TS_26',
 'TS_3',
 'TS_4',
 'TS_5',
 'TS_6',
 'TS_7',
 'TS_8',
 'TS_9']

In [9]:
util.get_run_topo(copickRoot, 'TS_0', voxel_size=10, algo="wbp")

CopickTomogram(tomo_type=wbp, len(features)=None) at 0x7fcf35bff760

In [6]:
util.get_run_obj_coords(copickRoot, 'TS_0')

[{'label': 1, 'x': 184, 'y': 201, 'z': 88, 'phi': 0, 'psi': 0, 'the': 0},
 {'label': 1, 'x': 174, 'y': 196, 'z': 97, 'phi': 0, 'psi': 0, 'the': 0},
 {'label': 1, 'x': 166, 'y': 203, 'z': 102, 'phi': 0, 'psi': 0, 'the': 0},
 {'label': 1, 'x': 159, 'y': 211, 'z': 93, 'phi': 0, 'psi': 0, 'the': 0},
 {'label': 1, 'x': 154, 'y': 222, 'z': 103, 'phi': 0, 'psi': 0, 'the': 0},
 {'label': 1, 'x': 165, 'y': 216, 'z': 104, 'phi': 0, 'psi': 0, 'the': 0},
 {'label': 1, 'x': 173, 'y': 225, 'z': 106, 'phi': 0, 'psi': 0, 'the': 0},
 {'label': 1, 'x': 182, 'y': 215, 'z': 101, 'phi': 0, 'psi': 0, 'the': 0},
 {'label': 1, 'x': 184, 'y': 204, 'z': 100, 'phi': 0, 'psi': 0, 'the': 0},
 {'label': 1, 'x': 492, 'y': 447, 'z': 169, 'phi': 0, 'psi': 0, 'the': 0},
 {'label': 1, 'x': 479, 'y': 443, 'z': 162, 'phi': 0, 'psi': 0, 'the': 0},
 {'label': 1, 'x': 470, 'y': 441, 'z': 154, 'phi': 0, 'psi': 0, 'the': 0},
 {'label': 1, 'x': 479, 'y': 440, 'z': 146, 'phi': 0, 'psi': 0, 'the': 0},
 {'label': 1, 'x': 485, 'y':

# plot

In [9]:
import util
import zarr
copickRoot = util.get_copick('coi', 'train')


In [13]:
run = 'TS_6_4'
algo = 'wbp'
topo = util.get_run_topo(copickRoot, run, voxel_size=10, algo=algo)
topo = zarr.open(topo.zarr())[0][:]
target = util.get_run_target(copickRoot, run, topo)

ModuleNotFoundError: No module named 'mrcfile'

In [ ]:


plt.figure(figsize=(15, 5))

# Original Image
plt.subplot(1, 2, 1)
plt.title('Tomogram')
plt.imshow(train_tomogram[90,],cmap='gray')
plt.axis('off')

# Original Image
plt.subplot(1, 2, 2)
plt.title('Train Target')
plt.imshow(train_target[90,])
plt.axis('off')

plt.tight_layout()
plt.show()